In [1]:
!nvcc --version
!nvidia-smi
import torch
torch.__version__

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Mon May 13 16:52:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   31C    P8              11W /  7

'2.2.1+cu121'

In [2]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git@5aeb252b194b93dc2879b4ac34bc51a31b5aee13'

  Cloning https://github.com/facebookresearch/detectron2.git (to revision 5aeb252b194b93dc2879b4ac34bc51a31b5aee13) to /tmp/pip-req-build-py9c9yse
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-py9c9yse
  Running command git rev-parse -q --verify 'sha^5aeb252b194b93dc2879b4ac34bc51a31b5aee13'
  Running command git fetch -q https://github.com/facebookresearch/detectron2.git 5aeb252b194b93dc2879b4ac34bc51a31b5aee13
  Running command git checkout -q 5aeb252b194b93dc2879b4ac34bc51a31b5aee13
  Resolved https://github.com/facebookresearch/detectron2.git to commit 5aeb252b194b93dc2879b4ac34bc51a31b5aee13
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 10.1 MB/s eta 

In [3]:
%cd /content/
!git clone https://github.com/DomMcOyle/TACO-expl.git
%cd /content/TACO-expl
!git checkout maskdino
!git pull origin maskdino
%cd /content/TACO-expl/MaskDINO
!pip install -r requirements.txt
%cd /content/TACO-expl/MaskDINO/maskdino/modeling/pixel_decoder/ops
!sh make.sh

/content
Cloning into 'TACO-expl'...
remote: Enumerating objects: 2316, done.
remote: Counting objects: 100% (2316/2316), done.
remote: Compressing objects: 100% (1109/1109), done.
remote: Total 2316 (delta 1212), reused 2244 (delta 1168), pack-reused 0
Receiving objects: 100% (2316/2316), 130.59 MiB | 19.98 MiB/s, done.
Resolving deltas: 100% (1212/1212), done.
/content/TACO-expl
Branch 'maskdino' set up to track remote branch 'maskdino' from 'origin'.
Switched to a new branch 'maskdino'
From https://github.com/DomMcOyle/TACO-expl
 * branch            maskdino   -> FETCH_HEAD
Already up to date.
/content/TACO-expl/MaskDINO
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 3.0 MB/s eta 0:00:00
/content/TACO-expl/MaskDINO/maskdino/modeling/pixel_decoder/ops
running build
running build_py
creating build
creating build/lib.linux-x86_64-cpython-310
creating build/lib.linux-x86_64-cpython-310/functions
copying functions/ms_deform_attn_func.py -> build/lib.linux-x86_64-cpython-310/f

In [4]:
from google.colab import drive
drive.mount("/content/MyDrive/", force_remount = True)

Mounted at /content/MyDrive/


In [1]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.2 ; cuda:  cu121
detectron2: 0.6


In [2]:
%cd /content/
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

/content


In [3]:
import sys
sys.path.append("/content/TACO-expl/MaskDINO")
!cd /content/TACO-expl/MaskDINO/
cfg_biggest_path = '/content/TACO-expl/MaskDINO/configs/coco/instance-segmentation/swin/maskdino_R50_bs16_50ep_4s_dowsample1_2048.yaml'
model_biggest_weights_url = 'https://github.com/IDEA-Research/detrex-storage/releases/download/maskdino-v0.1.0/maskdino_swinl_50ep_300q_hid2048_3sd1_instance_maskenhanced_mask52.3ap_box59.0ap.pth'

cfg_smallest_path = '/content/TACO-expl/MaskDINO/configs/coco/instance-segmentation/maskdino_R50_bs16_50ep_3s.yaml'
model_smallest_weight_url = 'https://github.com/IDEA-Research/detrex-storage/releases/download/maskdino-v0.1.0/maskdino_r50_50ep_300q_hid1024_3sd1_instance_maskenhanced_mask46.1ap_box51.5ap.pth'

In [4]:
sys.path.append('/content/TACO-expl/MaskDINO/maskdino/')
from config import add_maskdino_config
%cd /content/TACO-expl/MaskDINO/maskdino/
from detectron2.projects.deeplab import add_deeplab_config
from maskdino import MaskDINO

def setup_cfg(cfg_path, weights_url):
    # load config from file and command-line arguments
    cfg = get_cfg()
    add_deeplab_config(cfg)
    add_maskdino_config(cfg)
    cfg.merge_from_file(cfg_path)
    #cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
    #cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.5
    cfg.MODEL.WEIGHTS = weights_url
    cfg.freeze()
    return cfg

cfg_alt_dino = setup_cfg(cfg_smallest_path, model_smallest_weight_url)
cfg_modified = cfg_alt_dino.copy()
print(cfg_modified["MODEL"])

/content/TACO-expl/MaskDINO/maskdino
ANCHOR_GENERATOR:
  ANGLES: [[-90, 0, 90]]
  ASPECT_RATIOS: [[0.5, 1.0, 2.0]]
  NAME: DefaultAnchorGenerator
  OFFSET: 0.0
  SIZES: [[32, 64, 128, 256, 512]]
BACKBONE:
  FREEZE_AT: 0
  NAME: build_resnet_backbone
DEVICE: cuda
FPN:
  FUSE_TYPE: sum
  IN_FEATURES: []
  NORM: 
  OUT_CHANNELS: 256
KEYPOINT_ON: False
LOAD_PROPOSALS: False
MASK_ON: False
META_ARCHITECTURE: MaskDINO
MaskDINO:
  BOX_LOSS: True
  BOX_WEIGHT: 5.0
  CLASS_WEIGHT: 4.0
  COST_BOX_WEIGHT: 5.0
  COST_CLASS_WEIGHT: 4.0
  COST_DICE_WEIGHT: 5.0
  COST_GIOU_WEIGHT: 2.0
  COST_MASK_WEIGHT: 5.0
  DEC_LAYERS: 9
  DEEP_SUPERVISION: True
  DICE_WEIGHT: 5.0
  DIM_FEEDFORWARD: 2048
  DN: seg
  DN_NOISE_SCALE: 0.4
  DN_NUM: 100
  DROPOUT: 0.0
  ENC_LAYERS: 0
  ENFORCE_INPUT_PROJ: False
  EVAL_FLAG: 1
  GIOU_WEIGHT: 2.0
  HIDDEN_DIM: 256
  IMPORTANCE_SAMPLE_RATIO: 0.75
  INITIALIZE_BOX_TYPE: bitmask
  INITIAL_PRED: True
  LEARN_TGT: False
  MASK_WEIGHT: 5.0
  NHEADS: 8
  NO_OBJECT_WEIGHT: 0.1


In [5]:
%cd /content/TACO-expl/
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

import os.path
import json
import argparse
import numpy as np
import random
import datetime as dt
import copy
import torch.nn as nn
from sklearn.model_selection import train_test_split
from pathlib import Path

import torch
import math
import torch.utils.data
from torch.utils.data import DataLoader
from pycocotools import mask as coco_mask


from torchvision.ops import RoIAlign
from HDDETR.datasets.torchvision_datasets.coco import CocoDetection as TvCocoDetection # from torchvision.datasets import CocoDetection as TvCocoDetection
                                                                                       # the right above import import does not work bc of attribute definitions
import HDDETR.datasets.transforms as T #from torchvision import transforms as T # this import does not contain the randomselect method
import HDDETR.util.misc as mutils

/content/TACO-expl


In [6]:
DEFAULT_RANDOM_SEED = 42
# basic random seed
def seedBasic(seed=DEFAULT_RANDOM_SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
# torch random seed
def seedTorch(seed=DEFAULT_RANDOM_SEED):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
# combine
def seedEverything(seed=DEFAULT_RANDOM_SEED):
    seedBasic(seed)
    seedTorch(seed)

seedEverything()

In [7]:
keep_categories = ["Bottle", "Bottle cap", "Can", "Cigarette", "Cup",
                   "Lid", "Plastic bag & wrapper", "Pop tab", "Straw"]

def create_map(original, keep_supercategories):
  class_map = {}
  for cat in original:
    if cat["supercategory"] in keep_supercategories:
      class_map[cat["name"]] = cat["supercategory"]
    else:
      class_map[cat["name"]] = "Other"
  return class_map

def replace_dataset_classes(dataset, class_map):
      """ Replaces classes of dataset based on a dictionary"""
      class_new_names = list(set(class_map.values()))
      class_new_names.sort()
      class_originals = copy.deepcopy(dataset['categories'])
      dataset['categories'] = []
      class_ids_map = {}  # map from old id to new id

      # Assign background id 0
      has_background = False
      if 'Background' in class_new_names:
          if class_new_names.index('Background') != 0:
              class_new_names.remove('Background')
              class_new_names.insert(0, 'Background')
          has_background = True

      # Replace categories
      for id_new, class_new_name in enumerate(class_new_names):
          # Make sure id:0 is reserved for background
          id_rectified = id_new
          if not has_background:
              id_rectified += 1

          category = {
              'supercategory': '',
              'id': id_rectified,  # Background has id=0
              'name': class_new_name,
          }
          dataset['categories'].append(category)
          # Map class names
          for class_original in class_originals:
              if class_map[class_original['name']] == class_new_name:
                  class_ids_map[class_original['id']] = id_rectified

      # Update annotations category id tag
      for ann in dataset['annotations']:
          ann['category_id'] = class_ids_map[ann['category_id']]

In [8]:
with open("/content/TACO-expl/data/annotations.json", "r") as f: #"/content/TACO/data/annotations.json"
    dataset = json.loads(f.read())

class_map = create_map(dataset["categories"], keep_categories)
replace_dataset_classes(dataset, class_map)
dataset["categories"]

[{'supercategory': '', 'id': 1, 'name': 'Bottle'},
 {'supercategory': '', 'id': 2, 'name': 'Bottle cap'},
 {'supercategory': '', 'id': 3, 'name': 'Can'},
 {'supercategory': '', 'id': 4, 'name': 'Cigarette'},
 {'supercategory': '', 'id': 5, 'name': 'Cup'},
 {'supercategory': '', 'id': 6, 'name': 'Lid'},
 {'supercategory': '', 'id': 7, 'name': 'Other'},
 {'supercategory': '', 'id': 8, 'name': 'Plastic bag & wrapper'},
 {'supercategory': '', 'id': 9, 'name': 'Pop tab'},
 {'supercategory': '', 'id': 10, 'name': 'Straw'}]

In [9]:
classes = [elem["name"] for elem in dataset["categories"]]
print(classes)

['Bottle', 'Bottle cap', 'Can', 'Cigarette', 'Cup', 'Lid', 'Other', 'Plastic bag & wrapper', 'Pop tab', 'Straw']


In [10]:
!cp -r /content/MyDrive/MyDrive/official/ /content/

In [10]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog

train_annotation_file = '/content/TACO-expl/data/annotations_off_0_train.json'
val_annotation_file = '/content/TACO-expl/data/annotations_off_0_val.json'

img_dir = '/content/official/'

register_coco_instances("TACO_train", {}, train_annotation_file, img_dir)
MetadataCatalog.get("TACO_train").set(thing_classes = classes)
dataset_dicts_train = DatasetCatalog.get("TACO_train")

register_coco_instances("TACO_val", {}, val_annotation_file, img_dir)
MetadataCatalog.get("TACO_val").set(thing_classes = classes)
dataset_dicts_val = DatasetCatalog.get("TACO_val")

[05/13 17:49:06 d2.data.datasets.coco]: Loaded 1200 images in COCO format from /content/TACO-expl/data/annotations_off_0_train.json
[05/13 17:49:06 d2.data.datasets.coco]: Loaded 150 images in COCO format from /content/TACO-expl/data/annotations_off_0_val.json


In [12]:
train_cfg = cfg_alt_dino.clone()
with open("/content/TACO-expl/MaskDINO/configs/taco/taco_train.yaml", "w") as f:
  f.write(train_cfg.dump())

In [13]:
!mkdir /content/TACO-expl/maskdino_config/
!mkdir /content/output/
!mkdir /content/output/chkpt/

mkdir: cannot create directory ‘/content/TACO-expl/maskdino_config/’: File exists


In [11]:
train_cfg_loaded = get_cfg()
train_cfg_loaded.set_new_allowed(True)
train_cfg_loaded.merge_from_file("/content/TACO-expl/maskdino_config/taco_train_maskdino.yaml")
print(train_cfg_loaded.MODEL.BACKBONE.FREEZE_AT)
print(train_cfg_loaded.SOLVER.IMS_PER_BATCH)
print(train_cfg_loaded.INPUT.MIN_SIZE_TRAIN)
print(train_cfg_loaded.DATASETS.PRECOMPUTED_PROPOSAL_TOPK_TRAIN)

5
4
(480, 512, 544, 576, 608, 640, 672, 704, 736, 768, 800)
100


In [12]:
%cd /content/TACO-expl/MaskDINO/
from train_net import Trainer
!python -m pip install numpy==1.23.1
import numpy as np
np.bool = np.bool_

/content/TACO-expl/MaskDINO


In [16]:
%cd /content/output/chkpt/

/content/output/chkpt


In [ ]:
trainer = Trainer(train_cfg_loaded)
trainer.build_hooks()
trainer.resume_or_load(resume = True)  # load last checkpoint or MODEL.WEIGHTS
trainer.train()

criterion.weight_dict  {'loss_ce': 4.0, 'loss_mask': 5.0, 'loss_dice': 5.0, 'loss_bbox': 5.0, 'loss_giou': 2.0, 'loss_ce_interm': 4.0, 'loss_mask_interm': 5.0, 'loss_dice_interm': 5.0, 'loss_bbox_interm': 5.0, 'loss_giou_interm': 2.0, 'loss_ce_dn': 4.0, 'loss_mask_dn': 5.0, 'loss_dice_dn': 5.0, 'loss_bbox_dn': 5.0, 'loss_giou_dn': 2.0, 'loss_ce_interm_dn': 4.0, 'loss_mask_interm_dn': 5.0, 'loss_dice_interm_dn': 5.0, 'loss_bbox_interm_dn': 5.0, 'loss_giou_interm_dn': 2.0, 'loss_ce_0': 4.0, 'loss_mask_0': 5.0, 'loss_dice_0': 5.0, 'loss_bbox_0': 5.0, 'loss_giou_0': 2.0, 'loss_ce_interm_0': 4.0, 'loss_mask_interm_0': 5.0, 'loss_dice_interm_0': 5.0, 'loss_bbox_interm_0': 5.0, 'loss_giou_interm_0': 2.0, 'loss_ce_dn_0': 4.0, 'loss_mask_dn_0': 5.0, 'loss_dice_dn_0': 5.0, 'loss_bbox_dn_0': 5.0, 'loss_giou_dn_0': 2.0, 'loss_ce_interm_dn_0': 4.0, 'loss_mask_interm_dn_0': 5.0, 'loss_dice_interm_dn_0': 5.0, 'loss_bbox_interm_dn_0': 5.0, 'loss_giou_interm_dn_0': 2.0, 'loss_ce_1': 4.0, 'loss_mask_1':

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[05/13 17:49:37 d2.utils.events]:  iter: 479  total_loss: 50.16  loss_ce: 1.52  loss_mask: 0.04044  loss_dice: 0.7049  loss_bbox: 0.04519  loss_giou: 0.1963  loss_ce_dn: 0.4521  loss_mask_dn: 0.04673  loss_dice_dn: 0.7847  loss_bbox_dn: 0.06342  loss_giou_dn: 0.2261  loss_ce_0: 2.033  loss_mask_0: 0.04422  loss_dice_0: 0.7107  loss_bbox_0: 0.05107  loss_giou_0: 0.2091  loss_ce_dn_0: 1.465  loss_mask_dn_0: 0.3635  loss_dice_dn_0: 2.423  loss_bbox_dn_0: 0.3454  loss_giou_dn_0: 0.8547  loss_ce_1: 1.943  loss_mask_1: 0.04743  loss_dice_1: 0.7525  loss_bbox_1: 0.06062  loss_giou_1: 0.229  loss_ce_dn_1: 0.532  loss_mask_dn_1: 0.04736  loss_dice_dn_1: 0.8472  loss_bbox_dn_1: 0.11  loss_giou_dn_1: 0.368  loss_ce_2: 1.777  loss_mask_2: 0.04422  loss_dice_2: 0.7581  loss_bbox_2: 0.04859  loss_giou_2: 0.2088  loss_ce_dn_2: 0.5442  loss_mask_dn_2: 0.04754  loss_dice_dn_2: 0.8035  loss_bbox_dn_2: 0.08335  loss_giou_dn_2: 0.2878  loss_ce_3: 1.572  loss_mask_3: 0.04024  loss_dice_3: 0.7684  loss_bbox

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[05/13 17:50:21 d2.utils.memory]: Attempting to copy inputs of <bound method MaskDINO.instance_inference of MaskDINO(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv3): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
         

# Run this if you want to check how many trainable parameters we have

In [ ]:
from detectron2.modeling import build_model
model = build_model(train_cfg_loaded)
num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(num_trainable_params)

criterion.weight_dict  {'loss_ce': 4.0, 'loss_mask': 5.0, 'loss_dice': 5.0, 'loss_bbox': 5.0, 'loss_giou': 2.0, 'loss_ce_interm': 4.0, 'loss_mask_interm': 5.0, 'loss_dice_interm': 5.0, 'loss_bbox_interm': 5.0, 'loss_giou_interm': 2.0, 'loss_ce_dn': 4.0, 'loss_mask_dn': 5.0, 'loss_dice_dn': 5.0, 'loss_bbox_dn': 5.0, 'loss_giou_dn': 2.0, 'loss_ce_interm_dn': 4.0, 'loss_mask_interm_dn': 5.0, 'loss_dice_interm_dn': 5.0, 'loss_bbox_interm_dn': 5.0, 'loss_giou_interm_dn': 2.0, 'loss_ce_0': 4.0, 'loss_mask_0': 5.0, 'loss_dice_0': 5.0, 'loss_bbox_0': 5.0, 'loss_giou_0': 2.0, 'loss_ce_interm_0': 4.0, 'loss_mask_interm_0': 5.0, 'loss_dice_interm_0': 5.0, 'loss_bbox_interm_0': 5.0, 'loss_giou_interm_0': 2.0, 'loss_ce_dn_0': 4.0, 'loss_mask_dn_0': 5.0, 'loss_dice_dn_0': 5.0, 'loss_bbox_dn_0': 5.0, 'loss_giou_dn_0': 2.0, 'loss_ce_interm_dn_0': 4.0, 'loss_mask_interm_dn_0': 5.0, 'loss_dice_interm_dn_0': 5.0, 'loss_bbox_interm_dn_0': 5.0, 'loss_giou_interm_dn_0': 2.0, 'loss_ce_1': 4.0, 'loss_mask_1':